In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from tqdm import tqdm

In [2]:
def average_pairwise_distance(input, input1):
    similarities = {}
    total_files = sum(len(files) for _, _, files in os.walk(input))
    with tqdm(total=total_files) as pbar:
        for root, dirs, files in os.walk(input):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                with open(file_path, 'rb') as f:
                    embeddings_matrix = pickle.load(f)
                with open('{}/{}'.format(input1, file_name), 'rb') as f1:
                    embeddings_matrix1 = pickle.load(f1)
                APD = 1-np.mean(np.triu(cosine_similarity(embeddings_matrix,embeddings_matrix1)))
                similarities[os.path.splitext(file_name)[0]] = APD
                pbar.update(1)
    return similarities

In [14]:
print(sorted(average_pairwise_distance('embeddingsgood', 'embeddingsgood1').items(), reverse=True, key=lambda x:x[1])[:100])

100%|██████████| 12869/12869 [00:04<00:00, 2710.33it/s]


0.5873401224177816


 53%|█████▎    | 6878/12869 [00:02<00:02, 2678.88it/s]


KeyboardInterrupt: 

In [3]:
import json
import random
p = 0
l = []
for i in sorted(average_pairwise_distance('embeddingsgood', 'embeddingsgood1').items(), reverse=True, key=lambda x:x[1]):
    if i[0].islower() and p < 101:
        p += 1
        with open('JSONLgood/{}.jsonl'.format(i[0]), 'r') as file:
            y = 0
            json_data = file.read()
            for e in json.loads(json_data):
                y += 1
                print(y, 't1', e['sentence'], e['lemma'])
        with open('JSONLgood1/{}.jsonl'.format(i[0]), 'r') as file:
            y = 0
            json_data1 = file.read()
            for e in json.loads(json_data1):
                y += 1
                print(y, 't2', e['sentence'], e['lemma'])

100%|██████████| 12869/12869 [00:04<00:00, 2692.51it/s]


1 t1 Ponieważ żaden z przedmówców moich szczegółowo nie zastanawiał się nad ustawą o środkach naprawy państwowej gospodarki finansowej, a wyjmowano z niej tylko pewne fragmenty i te omawiano, chciałbym kolejno nad artykułami tej ustawy tutaj się zastanowić i z naszego punktu widzenia oświetlić je. oświetlić
2 t1 Na ulicach, które miasta już urządziły, a które niejednokrotnie do tej pory nie mają uregulowanego tytułu własności, a są prywatne, zjawia się ni stąd ni zowąd właściciel, która powiada, że nie pozwala przeprowadzić na nich kanalizacji, czy oświetlenia, czy innego urządzenia użyteczności publicznej. oświetlić
3 t1 Jeżeli istotnie członkowie tego Wysokiego Sejmu co do poszczególnych artykułów mają coś do powiedzenia, to przecież po to, aby w odpowiedni ich zdaniem sposób oświetlić daną materię i spowodować ustosunkowanie się Sejmu wobec niej przez głosowanie. oświetlić
4 t1 Zaznaczam, że dotychczas była taka praktyka, że po ogłoszeniu wniosku o zamknięciu dyskusji, dyskusja się 

In [3]:
# import json
# import random
# p = 0
# l = []
# for i in sorted(average_pairwise_distance('embeddingsgood', 'embeddingsgood1').items(), reverse=True, key=lambda x:x[1]):
#     if i[0].islower() and p < 101:
#         p += 1
#         with open('JSONLgood/{}.jsonl'.format(i[0]), 'r') as file:
#             y = 0
#             json_data = file.read()
#             for e in random.sample(json.loads(json_data), 10):
#                 y += 1
#                 l.append(('t1', y, e['sentence'], e['lemma'], e['first_index'], e['last_index']))
#         with open('JSONLgood1/{}.jsonl'.format(i[0]), 'r') as file:
#             y = 0
#             json_data1 = file.read()
#             for e in random.sample(json.loads(json_data1), 10):
#                 y += 1
#                 l.append(('t2', y, e['sentence'], e['lemma'], e['first_index'], e['last_index']))
# with open('annotateAPD.pickle', 'wb') as file:
#     pickle.dump(l, file)

import json
import random
p = 0
l1 = []
for i in sorted(average_pairwise_distance('embeddingsgood', 'embeddingsgood1').items(), reverse=True, key=lambda x:x[1]):
    if i[0].islower() and p < 101:
        p += 1
        l = []
        lis = []
        with open('JSONLgood/{}.jsonl'.format(i[0]), 'r') as file:
            y = 0
            json_data = file.read()
            for e in random.sample(json.loads(json_data), 10):
                y += 1
                l.append((y, 't1', e['sentence'], e['lemma'], e['first_index'], e['last_index']))
        with open('JSONLgood1/{}.jsonl'.format(i[0]), 'r') as file:
            y = 0
            json_data1 = file.read()
            for e in random.sample(json.loads(json_data1), 10):
                y += 1
                lis.append((y, 't2', e['sentence'], e['lemma'], e['first_index'], e['last_index']))
        zipped = list(zip(l, lis))
        l1.append(zipped)

100%|██████████| 12869/12869 [00:04<00:00, 2646.65it/s]


In [4]:
import csv
column_headers = ['sentence_number_1', 'time_period_1', 'sentence_1', 'lemma_1', 'first_index_1', 'last_index_1', 'sentence_number_2', 'time_period_2', 'sentence_2', 'lemma_2', 'first_index_2', 'last_index_2']
with open('outputAPD.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(column_headers)
    for item in l1:
        for i in item:
            flat_list = []
            for tup in i:
                flat_list.extend(list(tup))
            writer.writerow(flat_list)